<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e4_make_errors_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install kiwipiepy
# # !pip install nltk
# # !pip install
# # !pip install selenium

# #코랩에서 구글드라이버 연동
# # !apt-get update
# # !apt install chromium-chromedriver

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


# 0.Data Load

In [3]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [4]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280765 entries, 0 to 585940
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  280765 non-null  object
 1   lawName   280765 non-null  object
 2   ENG       280765 non-null  object
 3   KOR       280765 non-null  object
dtypes: object(4)
memory usage: 10.7+ MB


In [489]:
df.loc[df.KOR.str.contains(pat)]

<ipython-input-489-442e5e2ac6a2>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df.KOR.str.contains(pat)]


,category,lawName,ENG,KOR
186,헌법,개성공업지구 지원에 관한 법률 시행령,For purposes of applying the subparagraphs of ...,법 제13조제1항 각 호 및 법 제15조제1항 각 호를 적용할 때에 상시근로자의 수...
275,헌법,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정,"The term ""written judgment"" means an original ...","""재판서""란 판결·결정·명령의 재판을 적은 문서로서 재판을 한 법관이 서명날인한 원..."
282,헌법,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정,Upon receiving an application for the issuance...,외교부장관 또는 법무부장관은 제4조에 따라 아포스티유 또는 본부영사확인서 발급 신청...
293,헌법,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정,"Notwithstanding paragraph (1), the Minister of...",제1항에도 불구하고 외교부장관 또는 법무부장관은 제11조에 따라 서명 또는 직인의 ...
300,헌법,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정,The Minister of Foreign Affairs or the Ministe...,외교부장관 및 법무부장관은 제3조에 따른 공문서의 아포스티유 및 본부영사확인서 발급...
...,...,...,...,...
585620,외무,재외공관 공증법 시행령,In preparing documents in a diplomatic mission...,"공관에서 공증에 관한 문서를 작성할 때에는 공관의 소재지 및 명칭을 적고, 공증담당..."
585660,외무,재외공관 공증법 시행령,"When a person prepares notarial deeds, notaria...","공정증서, 인증서, 확인서와 그 정본 및 등본을 작성하는 경우 작성자의 서명날인 용..."
585670,외무,재외공관 공증법 시행령,"Where an execution clause is granted, the pers...",집행문을 부여한 경우에는 공정증서 원본 작성자의 서명날인 부분 밑에 집행문을 부여하...
585687,외무,재외공관 공증법 시행령,In preparing notarial deeds by using document ...,별지 제23호서식 외의 서식을 사용하여 공정증서를 작성하는 경우에는 별지 제24호서...


# 1.Data Fiiltering

In [6]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [7]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [8]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [9]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
# df_samp = df.copy(deep=True)

In [224]:
df_samp = df.loc[df.lawName.str.contains('은행법|여신|보험|신용|금융')|df.category.str.contains('경제|금융|상업')]

In [225]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
# df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
df_samp["errLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

<ipython-input-225-e55a75fc50f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_samp["curHyp"] = df_samp.KOR.values
<ipython-input-225-e55a75fc50f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
<ipython-input-225-e55a75fc50f4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [226]:
df_samp.head()

,category,lawName,ENG,KOR,curHyp,errors,errLog
22614,형사법,보험사기방지 특별법 시행령,The purpose of this Decree is to prescribe mat...,이 영은 「보험사기방지 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을...,이 영은 「보험사기방지 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을...,[],{}
22615,형사법,보험사기방지 특별법 시행령,Reporting prescribed in Article 4 of the Speci...,"「보험사기방지 특별법」(이하 ""법""이라 한다) 제4조에 따른 보고는 금융위원회가 정...","「보험사기방지 특별법」(이하 ""법""이라 한다) 제4조에 따른 보고는 금융위원회가 정...",[],{}
22620,형사법,보험사기방지 특별법 시행령,"""Grounds prescribed by Presidential Decree"" in...","법 제5조제2항에서 ""대통령령으로 정하는 사유""란 다음 각 호의 어느 하나에 해당하...","법 제5조제2항에서 ""대통령령으로 정하는 사유""란 다음 각 호의 어느 하나에 해당하...",[],{}
22621,형사법,보험사기방지 특별법 시행령,Where the terms and conditions of the relevant...,해당 보험계약의 약관 또는 다른 법령에서 보험금 지급을 지체 또는 거절하거나 삭감하...,해당 보험계약의 약관 또는 다른 법령에서 보험금 지급을 지체 또는 거절하거나 삭감하...,[],{}
22622,형사법,보험사기방지 특별법 시행령,Where justifiable grounds exists to suspect th...,보험회사가 보험사고 조사 과정에서 법 제4조에 따른 보험계약자등의 행위가 보험 사기...,보험회사가 보험사고 조사 과정에서 법 제4조에 따른 보험계약자등의 행위가 보험 사기...,[],{}


**주요 함수**

In [438]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["errLog"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "errLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def dup_drop():
    for k in dindex_dict.keys():
        dindex_dict[k] = list(set(dindex_dict[k]))

def check_idx(df):
    dup_drop()
    for k in dindex_dict.keys():
        print(f"{k}: {len(dindex_dict[k])}")
    print("############################################")
    print("omission: ", df.loc[df.errLog.apply(lambda x: "omission" in x.keys())].shape[0])
    print("addition: ", df.loc[df.errLog.apply(lambda x: "addition" in x.keys())].shape[0])
    print("mis-translation: ", df.loc[df.errLog.apply(lambda x: "mis-translation" in x.keys())].shape[0])
    print("over-translation: ", df.loc[df.errLog.apply(lambda x: "over-translation" in x.keys())].shape[0])
    print("under-translation: ", df.loc[df.errLog.apply(lambda x: "under-translation" in x.keys())].shape[0])
    print("grammer: ", df.loc[df.errLog.apply(lambda x: "grammer" in x.keys())].shape[0])

def check_memorize(data):
    df = copy_df(data.loc[:, ["curHyp","errLog"]], col=["errLog"])
    result_list = []
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        d = df.loc[i, "errLog"]
        result_dict = dict()
        for k in d.keys():
            start = d[k]["start_idx"]
            end = start+len(d[k]["text"])
            result_dict[k] = cur_hyp[start:end]
        result_list.append(result_dict)
    df["validation"] = result_list
    return df

def move_index(df, idx, start, word):
    err_log = copy.deepcopy(df.loc[idx, "errLog"])
    loop_keys = err_log.keys()
    for k in loop_keys:
        pre_idx = err_log[k]["start_idx"]
        if pre_idx > start:
            df.loc[idx, "errLog"][k]["start_idx"] = pre_idx + (len(word)+1)

def replace_inteli(df, idx, keyword, rword):
    errLog = df.loc[idx ,"errLog"]
    cur_hyp = df.loc[idx, "curHyp"]

    text_dict=dict()
    for k in errLog.keys():
        if k=="omission": continue
        text, sindex = errLog[k]["text"], errLog[k]["start_idx"]
        text_dict[text] = (sindex, sindex+len(text))

    reject_range = []
    for key in text_dict.keys():
        if keyword in key:
            reject_range.append(text_dict[key])

    for f in re.finditer(keyword, cur_hyp):
        start, end = f.start(), f.end()
        is_reject = False
        for r_s, r_e in reject_range:
            if r_s <= start < r_e:
                is_reject=True
        if is_reject: continue
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        while end<len(cur_hyp) and cur_hyp[end]!=" ":
            end+=1
        start += 1

        word = cur_hyp[start:end]
        if word in text_dict.keys():
            start_idx = text_dict[word]
            if start==start_idx: continue
            break
        break
    add_word = word.replace(keyword, rword)
    conv_sentence = cur_hyp[:start]+add_word+cur_hyp[start+len(word):]
    return conv_sentence, start, add_word

def can_noise(df, i):
    cur_hyp = df.loc[i, "curHyp"]
    errLog = df.loc[i, "errLog"]
    for k in errLog.keys():
        if k=="omission": continue
        start, text = errLog[k]["start_idx"], errLog[k]["text"]
        end = start+len(text)
        cur_hyp = cur_hyp[:start]+("ㅱ"*len(text))+cur_hyp[end:]
    return cur_hyp

def errors_report(df, rate=False):
    sevs = {"critical":0, "major":1, "minor":2, "neutral":3}
    err_types = {
        'omission':[0, 0, 0, 0],
        'addition':[0, 0, 0, 0],
        'mis-translation':[0, 0, 0, 0],
        'over-translation':[0, 0, 0, 0],
        'under-translation':[0, 0, 0, 0],
        'grammer':[0, 0, 0, 0]
        }
    for i in df.index:
        errors = df.loc[i, "errors"]
        for d in errors:
            err_types[d["type"]][sevs[d["severity"]]]+=1
    report_df = pd.DataFrame(err_types, index=sevs.keys())
    report_df["sum"] = report_df.apply(np.sum, axis=1)
    report_df.loc["sum"] = report_df.apply(np.sum, axis=0)
    if rate:
        return_df = pd.DataFrame()
        for col in report_df.columns:
            return_df[col] = report_df[col].apply(lambda x: round((100*x)/report_df.loc["sum", "sum"], 2))
        return return_df
    else:
        return report_df

# 2.품사 태깅

In [228]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

22614     [(이, MM, 0, 1), (영, NNG, 2, 1), (은, JX, 3, 1),...
22615     [(「, SSO, 0, 1), (보험, NNG, 1, 2), (사기, NNG, 3,...
22620     [(법, NNG, 0, 1), (제, XPN, 2, 1), (5, SN, 3, 1)...
22621     [(해당, NNG, 0, 2), (보험, NNG, 3, 2), (계약, NNG, 5...
22622     [(보험, NNG, 0, 2), (회사, NNG, 2, 2), (가, JKS, 4,...
                                ...                        
577886    [(위원장, NNG, 0, 3), (은, JX, 3, 1), (법, NNG, 5, ...
577888    [(회의, NNG, 0, 2), (에, JKB, 2, 1), (참석, NNG, 4,...
577890    [(이, MM, 0, 1), (영, NNG, 2, 1), (에서, JKB, 3, 2...
577909    [(과학, NNG, 0, 2), (기술, NNG, 2, 2), (정보, NNG, 4...
577910    [(과학, NNG, 0, 2), (기술, NNG, 2, 2), (정보, NNG, 4...
Name: KOR, Length: 45093, dtype: object

In [229]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

22614     {'MM': ['이', '그'], 'NNG': ['영', '보험', '사기', '방...
22615     {'SSO': ['「', '(', '"'], 'NNG': ['보험', '사기', '...
22620     {'NNG': ['법', '항', '대통령령', '사유', '다음', '해당', '...
22621     {'NNG': ['해당', '보험', '계약', '약관', '법령', '보험금', ...
22622     {'NNG': ['보험', '회사', '보험', '사고', '조사', '과정', '...
                                ...                        
577886    {'NNG': ['위원장', '법', '분쟁', '조정', '결과', '분쟁', '...
577888    {'NNG': ['회의', '참석', '위원', '항', '회의', '출석', '의...
577890    {'MM': ['이'], 'NNG': ['영', '규정', '사항', '분쟁', '...
577909    {'NNG': ['과학', '기술', '정보', '통신부', '장관', '법', '...
577910    {'NNG': ['과학', '기술', '정보', '통신부', '장관', '법', '...
Name: KOR, Length: 45093, dtype: object

In [230]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

5261

In [231]:
tag = "NNG"
nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))
len(nouns)

5261

# 3.Error 생성

In [289]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}

## Omission

### Omission - Critical

In [290]:
cr_om_index = random_index(df_samp.curHyp, 0.02, 10)
# df_samp.loc[cr_om_index]

In [291]:
def critical_omission(df, seed, ratio):
    df_t = copy_df(df)
    cr_om_index = random_index(df_t.curHyp, ratio, seed)
    dindex_dict["omission"]+=list(cr_om_index)
    result = df_t.loc[cr_om_index]
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(seed)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in cr_om_index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")
        result.loc[i, "errLog"]["omission"] = {"start_idx": sidx, "text": reduce_sentence}

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return convert_df(df, result)

df_crom = critical_omission(df_samp, 10, 0.02)

In [292]:
df_crom.loc[cr_om_index].shape

(901, 7)

In [293]:
# df_crom.loc[cr_om_index]

In [294]:
check_idx(df_crom)

omission: 901
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  901
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


### Omission - Major

In [295]:
np.random.seed(20)
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
np.random.seed(20)
change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
chv_index = random_index(ch6, 0.65, 20)

In [296]:
def major_omission(df_local, seed, tag, slen, ratio):
    df_t = copy_df(df_local)
    np.random.seed(seed)
    nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
    np.random.seed(seed)
    change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
    ch6 = change_val[change_val.apply(lambda x: len(x[0])) > slen]
    chv_index = random_index(ch6, ratio, seed)
    # drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
    ch_srs = ch6.loc[chv_index]
    result = df_t.loc[chv_index]

    for i in chv_index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            dindex_dict["omission"]+=[i]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
            result.loc[i, "errLog"]["omission"] = {"start_idx": start, "text": del_unit}
        except:
            pass

    return convert_df(df_local, result)

df_maom = major_omission(df_crom, 20, "NNP", 3, 0.65)
# df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [297]:
df_maom.loc[df_maom.errors.apply(len)==1].shape[0]

3985

In [298]:
# df_maom.loc[df_maom.errors.apply(len)==1]

In [299]:
check_idx(df_maom)

omission: 3985
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  3985
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


In [300]:
df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [301]:
check_idx(df_mo)

omission: 6462
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  6462
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


In [302]:
# df_mo.loc[df_mo.errors.apply(len)==1]

### Omission - Neutral

In [303]:
sp_list = [".", ";", ":"]
np.random.shuffle(sp_list)
sp_list

['.', ';', ':']

In [304]:
test = "안녕하세요; 반갑습니다."
a = test.find(".")
test[:a] + test[a+1:]

'안녕하세요; 반갑습니다'

In [305]:
def omission_nt(df, ratio, seed):    # 0.01, 111
    df_t = copy_df(df)
    np.random.seed(seed)
    on_index = random_index(df_t.drop(index=dindex_dict["omission"]), ratio, seed)
    on_index = df_t.loc[on_index][df_t.loc[on_index].curHyp.str.contains(r"\.|\;|\:")].index
    result = df_t.loc[on_index]
    dindex_dict["omission"]+=list(on_index)

    sp_list = [".", ";", ":"]
    for i in on_index:
        np.random.shuffle(sp_list)
        for s in sp_list:
            cur_hyp = result.loc[i, "curHyp"]
            s_loc = cur_hyp.find(s)
            if s_loc==-1:
                continue
            conv_hyp = cur_hyp[:s_loc]+cur_hyp[s_loc+1:]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "neutral", s_loc, s, "omission")
            result.loc[i, "errLog"]["omission"] = {"start_idx": s_loc, "text": s}
            break
    return convert_df(df, result)

df_on = omission_nt(df_mo, 0.01, 111)

In [306]:
# df_on.loc[df_on.errors.apply(lambda x: x[0]["severity"]=="neutral" if len(x)!=0 else False)]

In [307]:
check_idx(df_on)

omission: 6788
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  6788
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


## Addition

###Addition-Major

In [308]:
ma_ad_index = random_index(df_on, 0.22, 55)
nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
nnpg_info.drop(index=drop_index, axis=0, inplace=True)
maad_index = list(set(ma_ad_index)-set(drop_index))

In [309]:
def major_addition(df, ratio, seed):
    np.random.seed(seed)
    df_t=copy_df(df)

    ma_ad_index = random_index(df_t, ratio, seed)
    nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
    nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
    drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
    nnpg_info.drop(index=drop_index, axis=0, inplace=True)
    ma_ad_index = list(set(ma_ad_index)-set(drop_index))
    np.random.seed(seed)
    add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
    np.random.seed(seed)
    add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
    result = df_t.loc[ma_ad_index]

    # dindex_dict["addition"]+=ma_ad_index

    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in ma_ad_index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        if start < 0:
            conv_hyp = f"{add_word}"+" "+cur_hyp[start+1:].strip()
        else:
            conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        move_index(result, i, start+1, add_word)
        result.loc[i, "errLog"]["addition"] = {"start_idx": (start+1), "text": add_word}
        dindex_dict["addition"].append(i)

    return convert_df(df, result)

df_maad = major_addition(df_on, 0.22, 55)

In [310]:
temp = check_memorize(df_maad.loc[maad_index])
ttp = temp.loc[temp.errLog.apply(lambda i: "addition" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["addition"]["text"]) != ttp["validation"].apply(lambda y: y["addition"])]

,curHyp,errLog,validation


In [311]:
check_idx(df_maad)

omission: 6788
addition: 9727
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  6788
addition:  9727
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


### Addition-Minor

In [312]:
amindex = random_index(df_maad.drop(index=dindex_dict["addition"]), 0.2, 234)

In [313]:
def ma_iter(df, ratio, seed):   #0.2 234
    df_t = copy_df(df)
    np.random.seed(seed)
    am_index = random_index(df_t.drop(index=dindex_dict["addition"]), ratio, seed)
    result = df_t.loc[am_index]
    dindex_dict["addition"]+=list(am_index)

    for i in am_index:
        cur_hyp = result.loc[i, "curHyp"]
        sch = cur_hyp.split(" ")
        rand_idx = np.random.choice([i for i in range(len(sch))])
        random_dup = sch[rand_idx]
        conv_hyp=" ".join(sch[:rand_idx] + [random_dup] +sch[rand_idx:])
        if rand_idx==0:
            start = 0
        else:
            start = len(" ".join(sch[:rand_idx]))+1

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "minor", start, random_dup, "addition")

        move_index(result, i, start, random_dup)
        result.loc[i, "errLog"]["addition"] = {"start_idx": start, "text": random_dup}

    return convert_df(df, result)
df_miter = ma_iter(df_maad, 0.2, 234)

In [314]:
temp = check_memorize(df_miter.loc[amindex])
ttp = temp.loc[temp.errLog.apply(lambda i: "addition" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["addition"]["text"]) != ttp["validation"].apply(lambda y: y["addition"])]

,curHyp,errLog,validation


In [315]:
check_idx(df_miter)

omission: 6788
addition: 16800
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  6788
addition:  16800
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


## Mis-Translation

In [443]:
dindex_dict["mis-translation"] = []

### MT - Minor

In [445]:
te_drop = df_miter.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
transHyp

22614     이 영은 ㅱㅱㅱㅱ 「보험사기방지 특별법」에서 위임된 사항과 그 시행에 필요한 사항을...
22615     「보험사기방지 특별법」(이하 "법"이라 한다) 제4조에 따른 보고는 금융위원회가 정...
22620     법 제5조제2항에서 "대통령령으로 정하는 사유"란 다음 각 호의 어느 하나에 ㅱㅱㅱ...
22621     해당 보험계약의 약관 또는 다른 법령에서 보험금 지급을 지체 또는 거절하거나 삭감하...
22622     보험회사가 보험사고 조사 과정에서 법 제4조에 따른 보험계약자등의 행위가 보험 사기...
                                ...                        
577886    위원장은 법 제48조의5에 따른 분쟁조정 결과 또는 분쟁조정 회의에 부치지 아니하기...
577888    회의에 참석하는 위원 및 제7조제1항에 따라 회의에 출석하여 의견을 진술하는 전문가...
577890    이 영에서 규정한 사항 외에 분쟁조정위원회의 운영 등에 필요한 사항은 분쟁조정위원회...
577909    ㅱㅱㅱ 과학기술정보통신부장관은 법 제51조에 따라 다음 각 호의 권한을 우정사업본부...
577910    과학기술정보통신부장관은 법 제51조에 따라 법 제18조에 따른 예금자금 중 과학기술...
Length: 45093, dtype: object

In [446]:
te_drop = df_miter.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
temp = te_drop.loc[transHyp.str.contains(r'하여야 한다')]
can_idx = random_index(temp, 0.3, 789)

In [450]:
def hv_mistrans(df, ratio, seed):   # 0.3 789
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.str.contains(r'하여야 한다')]
    can_index = random_index(temp, ratio, seed)
    result = te_drop.loc[can_index]
    ch_words = ['해야만 한다', '해야 할 것이다']
    dindex_dict["mis-translation"] += list(can_index)

    for i in can_index:
        np.random.shuffle(ch_words)
        ch_word = ch_words[0]
        cur_hyp = result.loc[i, "curHyp"]
        start = cur_hyp.find('하여야 한다')
        conv_hyp = cur_hyp[:start] + ch_word + cur_hyp[start+6:]

        result.loc[i, 'curHyp'] = conv_hyp
        input_errors(result, i, 'minor', start, ch_word, 'mis-translation')

        move_index(result, i, start, ch_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": ch_word}
    return convert_df(df, result)

df_can = hv_mistrans(df_miter, 0.3, 789)

In [451]:
temp = check_memorize(df_can.loc[can_idx])
temp
ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

,curHyp,errLog,validation


In [452]:
check_idx(df_can)

omission: 6788
addition: 16800
mis-translation: 2736
over-translation: 0
under-translation: 0
grammer: 17313
############################################
omission:  6788
addition:  16800
mis-translation:  2736
over-translation:  0
under-translation:  0
grammer:  0


In [454]:
errors_report(df_can, True)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,3.42,0.00,0.00,0.0,0.0,0.0,3.42
major,21.13,36.95,0.00,0.0,0.0,0.0,58.08
minor,0.00,26.87,10.39,0.0,0.0,0.0,37.26
neutral,1.24,0.00,0.00,0.0,0.0,0.0,1.24
sum,25.79,63.82,10.39,0.0,0.0,0.0,100.00


In [461]:
def ddo_trans(df, ratio, seed, val):
    df_t = copy_df(df)
    conv_list = ["내지", "및", "또는"]
    conv_list.remove(val)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    # temp = df_t.drop(index=dindex_dict["mis-translation"])
    ival = " "+val+" "
    ddo_idx = random_index(te_drop.loc[transHyp.str.contains(ival)], ratio, seed)
    result = te_drop.loc[ddo_idx]
    dindex_dict["mis-translation"]+=list(ddo_idx)


    for i in ddo_idx:
        np.random.shuffle(conv_list)
        conv_word = conv_list[0]

        conv_hyp, start, fconv_word = replace_inteli(result, i, val, conv_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, fconv_word, 'mis-translation')

        move_index(result, i, start, fconv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": fconv_word}
    return convert_df(df, result)

In [462]:
te_drop = df_can.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
do_idx = random_index(te_drop.loc[transHyp.str.contains(" "+"또는"+" ")], 0.2, 555)

df_ddo = ddo_trans(df_can, 0.2, 555, "또는")

temp = check_memorize(df_ddo.loc[do_idx])
ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

,curHyp,errLog,validation


In [463]:
check_idx(df_ddo)

omission: 6788
addition: 16800
mis-translation: 4669
over-translation: 0
under-translation: 0
grammer: 17313
############################################
omission:  6788
addition:  16800
mis-translation:  4669
over-translation:  0
under-translation:  0
grammer:  0


In [465]:
te_drop = df_ddo.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
do_idx = random_index(te_drop.loc[transHyp.str.contains(" "+"내지"+" ")], 0.25, 444)

df_nae = ddo_trans(df_ddo, 0.25, 444, "내지")

temp = check_memorize(df_nae.loc[do_idx])
ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

,curHyp,errLog,validation


In [466]:
te_drop = df_nae.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
do_idx = random_index(te_drop.loc[transHyp.str.contains(" "+"및"+" ")], 0.3, 333)

df_mit = ddo_trans(df_nae, 0.3, 333, "및")

temp = check_memorize(df_mit.loc[do_idx])
ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

,curHyp,errLog,validation


In [467]:
check_idx(df_mit)

omission: 6788
addition: 16800
mis-translation: 7591
over-translation: 0
under-translation: 0
grammer: 17313
############################################
omission:  6788
addition:  16800
mis-translation:  7591
over-translation:  0
under-translation:  0
grammer:  0


In [ ]:
# def standard_change(df, ratio, seed, val, errType):
#     df_t = copy_df(df)
#     np.random.seed(seed)
#     std_idx = random_index(df_t.drop(index=dindex_dict[errType]))

### MT - Major

In [ ]:
re.split(r"[^\d]+",'제16조의10제3항')

['', '16', '10', '3', '']

In [470]:
patterns = r"제[\d]+[\w]+[\d]+[\w]+\s"
compiler = re.compile(patterns)
te_drop = df_mit.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns)], 0.35, 987)
result = te_drop.loc[std_idx]

for i in std_idx:
    cur_hyp = result.loc[i, "curHyp"]
    all_find = list(compiler.finditer(cur_hyp))
    np.random.shuffle(all_find)
    for f in all_find:
        sts = f.group().strip()
        sts_nsp = re.sub(r"[가-힣]+", " ", sts)
        sts_list = re.split(r"[^\d]+", sts_nsp)[1:-1]
        sts_add = re.split(r"[\d]+", sts)
        pre_shuffle = copy.deepcopy(sts_list)
        if len(list(set(pre_shuffle)))==1: continue
        while True:
            np.random.shuffle(pre_shuffle)
            if sts_list!=pre_shuffle:
                break
        conv_hyp = sts_add.pop(0)
        for a, b in zip(sts_add, pre_shuffle):
            conv_hyp+=(b+a)
        break
    print(cur_hyp, sts)
    print(replace_inteli(result, i, sts, conv_hyp))
    break


조달청장은 법 제7조제3항제4호에 따라 각 중앙관서의 장의 물품관리 상황을 조달청장이 정하는 기준에 따라 평가하여야 한다. 제7조제3항제4호에
('조달청장은 법 제7조제4항제3호에 따라 각 중앙관서의 장의 물품관리 상황을 조달청장이 정하는 기준에 따라 평가하여야 한다.', 8, '제7조제4항제3호에')


In [477]:
def replace_numbers(df, ratio, seed):
    df_t = copy_df(df)
    patterns = r"제[\d]+[\w]+[\d]+[\w]+\s"
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns)], ratio, seed)
    result = te_drop.loc[std_idx]

    for i in std_idx:
        cur_hyp = result.loc[i, "curHyp"]
        all_find = list(compiler.finditer(cur_hyp))
        np.random.shuffle(all_find)
        for f in all_find:
            sts = f.group().strip()
            sts_nsp = re.sub(r"[가-힣]+", " ", sts)
            sts_list = re.split(r"[^\d]+", sts_nsp)[1:-1]
            sts_add = re.split(r"[\d]+", sts)
            pre_shuffle = copy.deepcopy(sts_list)
            if len(list(set(pre_shuffle)))==1: continue
            while True:
                np.random.shuffle(pre_shuffle)
                if sts_list!=pre_shuffle:
                    break
            chyp = sts_add.pop(0)
            for a, b in zip(sts_add, pre_shuffle):
                chyp+=(b+a)

            conv_hyp, start, conv_word = replace_inteli(result, i, sts, chyp)
            break

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'major', start, conv_word, 'mis-translation')

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": conv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

df_rpnm = replace_numbers(df_mit, .35, 987)

In [478]:
temp = check_memorize(df_rpnm.loc[std_idx])
ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

,curHyp,errLog,validation


In [479]:
check_idx(df_rpnm)

omission: 6788
addition: 16800
mis-translation: 12270
over-translation: 0
under-translation: 0
grammer: 17313
############################################
omission:  6788
addition:  16800
mis-translation:  12270
over-translation:  0
under-translation:  0
grammer:  0


In [480]:
errors_report(df_rpnm, True)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,2.51,0.00,0.00,0.0,0.0,0.0,2.51
major,15.51,27.13,13.05,0.0,0.0,0.0,55.68
minor,0.00,19.73,21.17,0.0,0.0,0.0,40.89
neutral,0.91,0.00,0.00,0.0,0.0,0.0,0.91
sum,18.93,46.85,34.22,0.0,0.0,0.0,100.00


## Over / Under Translation

- 준용하다 -> 적용하다 : 과잉번역
- 적용하다 -> 준용하다 : 과소번역

In [437]:
import pickle
with open(os.path.join(gdrive_path, "etc_files/crawl_dict.pkl"), "rb") as f:
    term_dict = pickle.load(f)
# term_dict

## Grammer

### Grammer-Minor

In [493]:
dindex_dict["grammer"] = []

In [494]:
jsg_index = random_index(df_rpnm, 0.2, 1010)
jsg_tokens = df_rpnm.loc[jsg_index].curHyp.apply(lambda x: kiwi.tokenize(x))

In [495]:
jsd = {
        "가":"이", "을":"를", "로써":"으로써", "로서":"으로서",
        "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나",
        "와":"과", "랑":"이랑", "이랑":"랑", "며":"이며", "이며":"며"
        }
keys = list(jsd.keys())
for key in keys:
    jsd[jsd[key]] = key

need_josa = None
for i, tag in enumerate(["JKS", "JKC", "JKO", "JKB", "JX", "JC"]):
    if i==0:
        need_josa = jsg_tokens.apply(lambda x: find_tokens(x, tag))
    else:
        need_josa += jsg_tokens.apply(lambda x: find_tokens(x, tag))
drop_index = need_josa[need_josa.apply(len)==0].index
jsg_index = list(set(jsg_index)-set(drop_index))

In [496]:
def josa_mistranslation(df, ratio, seed):
    df_t = copy_df(df)
    js_mstr_index = random_index(df_t.drop(index=dindex_dict["grammer"]), ratio, seed)
    js_tokens = df_t.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))

    jsd = {
        "가":"이", "을":"를", "로써":"으로써", "로서":"으로서",
        "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나",
        "와":"과", "랑":"이랑", "이랑":"랑", "며":"이며", "이며":"며"
        }
    keys = list(jsd.keys())
    for key in keys:
        jsd[jsd[key]] = key

    tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
    need_josa = None
    for i, tag in enumerate(tags):
        if i==0:
            need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
        else:
            need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

    drop_index = need_josa[need_josa.apply(len)==0].index
    js_mstr_index = list(set(js_mstr_index)-set(drop_index))
    need_josa.drop(index=drop_index, axis=0, inplace=True)
    josa_elem = need_josa.apply(lambda x: list(set([val[0] for val in x])))
    result = df_t.loc[js_mstr_index]

    for i in js_mstr_index:
        cur_hyp = result.loc[i, "curHyp"]
        target = josa_elem[i]
        np.random.shuffle(target)
        for word in target:
            try:
                conv_hyp, start, conv_word = replace_inteli(result, i, word, jsd[word])
                result.loc[i, "curHyp"] = conv_hyp
                input_errors(result, i, 'minor', start, conv_word, 'grammer')

                move_index(result, i, start, conv_word)
                result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}
                dindex_dict["grammer"].append(i)
                break
            except:
                pass

    return convert_df(df, result)

df_jmt = josa_mistranslation(df_rpnm, 0.2, 1010)

In [497]:
temp = check_memorize(df_jmt.loc[jsg_index])
temp
ttp = temp.loc[temp.errLog.apply(lambda i: "grammer" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["grammer"]["text"]) != ttp["validation"].apply(lambda y: y["grammer"])]

,curHyp,errLog,validation


In [498]:
check_idx(df_jmt)

omission: 6788
addition: 16800
mis-translation: 12270
over-translation: 0
under-translation: 0
grammer: 8792
############################################
omission:  6788
addition:  16800
mis-translation:  12270
over-translation:  0
under-translation:  0
grammer:  8792


In [499]:
errors_report(df_jmt, True)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,2.02,0.00,0.00,0.0,0.0,0.00,2.02
major,12.45,21.78,10.48,0.0,0.0,0.00,44.72
minor,0.00,15.84,17.00,0.0,0.0,19.69,52.53
neutral,0.73,0.00,0.00,0.0,0.0,0.00,0.73
sum,15.20,37.63,27.48,0.0,0.0,19.69,100.00


In [506]:
te_drop = df_jmt.drop(index=dindex_dict["grammer"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
temp = te_drop.loc[transHyp.apply(lambda x: "한다" in x)]
handa_index = random_index(temp, 0.4, 141)

In [507]:
def hada_gram(df, ratio, seed):
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["grammer"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.apply(lambda x: "한다" in x)]
    handa_index = random_index(temp, 0.4, 141)
    result = df_t.loc[handa_index]
    replace_list = ["하다", "함"]

    for i in handa_index:
        dindex_dict["grammer"].append(i)
        cur_hyp = result.loc[i, "curHyp"]
        r_word = np.random.choice(replace_list)
        conv_hyp, start, conv_word = replace_inteli(result, i, "한다", r_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, conv_word, 'grammer')

        move_index(result, i, start, conv_word)
        result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}

    return convert_df(df, result)

df_hada = hada_gram(df_jmt, 0.4, 141)

In [508]:
temp = check_memorize(df_hada.loc[handa_index])
temp
ttp = temp.loc[temp.errLog.apply(lambda i: "grammer" in i)]
ttp.loc[ttp["errLog"].apply(lambda x: x["grammer"]["text"]) != ttp["validation"].apply(lambda y: y["grammer"])]

,curHyp,errLog,validation


In [509]:
check_idx(df_hada)

omission: 6788
addition: 16800
mis-translation: 12270
over-translation: 0
under-translation: 0
grammer: 17308
############################################
omission:  6788
addition:  16800
mis-translation:  12270
over-translation:  0
under-translation:  0
grammer:  17308


In [510]:
errors_report(df_hada, True)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,1.69,0.0,0.00,0.0,0.0,0.00,1.69
major,10.46,18.3,8.80,0.0,0.0,0.00,37.56
minor,0.00,13.3,14.28,0.0,0.0,32.55,60.14
neutral,0.61,0.0,0.00,0.0,0.0,0.00,0.61
sum,12.77,31.6,23.08,0.0,0.0,32.55,100.00


In [435]:
# df_hada.loc[df_hada.curHyp.apply(lambda x: len(x.split("."))) > 2]

# 후처리

In [ ]:
def convert_index(df_new):
    df = copy_df(df_new)
    new_errs = []
    for i in df.index:
        errors, errLog = df.loc[i, "errors"], df.loc[i, "errLog"]

        for i, err in enumerate(df.loc[i, "errors"]):
            errors[i]['start_idx'] = errLog[err["type"]]["start_idx"]

        new_errs.append(errors)
    df["errors"] = new_errs
    return df
temp = convert_index(df_mit)

In [87]:
# df_mit.loc[df_jmt.errLog.apply(lambda x: len(x)>2)].iloc[:10]

In [86]:
# temp.loc[df_jmt.errLog.apply(lambda x: len(x)>2)].iloc[:10]

In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_can)

In [ ]:
with open(os.path.join(gdrive_path, "data_0825.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)